In [1]:
#Import statements
import requests
import json
import pandas as pd
import pybomb as pb
import pybomb.clients.base_client as baseclient
import pybomb.clients.games_client as gamesclient
import pybomb.clients.game_client as gc
import numpy as np

1) get user input
2) search for the game
3) get the game id with games_client
4) send the game id to game_client to get the similar_games
5) recommend those games

rating, id, platform, genre

With Gameclient, you can get genre given id, similar_games
with gamesclient, get id

In [34]:
#Function Defitions:

def returnIds():

    #The API searches its database for every match to the api call
    #When the user enters the name of a game, it returns info about multiple games
    #This function returns just the id of the first game that matches and stores it in variable ids.
    
    user_game = input("Enter the name of a game you'd like to search for:")
    key = '64ce54b0ab6eb483b1964ae7979124f96aa791e5' ##Make this an input or encrypt it
    games_client = pb.GamesClient(key)
    
    try:
        response = games_client.quick_search(name = user_game)
        results = response.results

        name_list = []
        id_list = []

        for i in results:

            name_list.append(i['aliases'])
            id_list.append(i['id'])
        
        return(id_list[0])
        
    except (pb.exceptions.BadRequestException, IndexError, requests.exceptions.HTTPError, TypeError) as e:
        print('game not found')



def GetSimilarGames(game_id):
    
    #Gets a list of the names and urls of each recommended game
    key = '64ce54b0ab6eb483b1964ae7979124f96aa791e5'
    game_client = pb.GameClient(key, default_format = 'json')
    response = game_client.fetch(game_id)

    name_list_sim = []
    url_list_sim = []
    genre_list = []
    
    try:
        #for each id, append the names of reccommended games to a list.
        for i in response.results['similar_games']:

            name_list_sim.append(i['name'])
            url_list_sim.append(i['site_detail_url'])

        #return each list of reccommended games
        return(name_list_sim, url_list_sim, genre_list)
    except (pb.exceptions.BadRequestException, IndexError, requests.exceptions.HTTPError) as e:
        pass
def compileDF(recinfo):
    games = pd.Series(recinfo[0], name = 'Games')
    urls = pd.Series(recinfo[1], name = 'URLs')
    
    df = pd.DataFrame({'Games': games, 'URLs': urls})
    
    return(df)


In [4]:
#Testing Functions:

def GetSimilarGames(game_id):
    key = '64ce54b0ab6eb483b1964ae7979124f96aa791e5'
    game_client = pb.GameClient(key, default_format = 'json')
    response = game_client.fetch(game_id)
    name_list_sim = []
    id_list_sim= []
    
    #Gets the similar games out of the response
    return(response.results['similar_games'])

#At attempt at iterating through each id that's returned by the api, to get multiple databases.
#This didn't work out, so we went with just id[0].

    #for i in response.results['similar_games']:
        #name_list_sim.append(i['name'])
        #print(i)
        #id_list_sim.append(i['id'])
    #print(name_list_sim)
    #print(id_list_sim)

id = 29903
games= GetSimilarGames(id)
games

[{'api_detail_url': 'https://www.giantbomb.com/api/game/3030-12231/',
  'id': 12231,
  'name': 'Gears of War',
  'site_detail_url': 'https://www.giantbomb.com/gears-of-war/3030-12231/'},
 {'api_detail_url': 'https://www.giantbomb.com/api/game/3030-20662/',
  'id': 20662,
  'name': 'Gears of War 2',
  'site_detail_url': 'https://www.giantbomb.com/gears-of-war-2/3030-20662/'},
 {'api_detail_url': 'https://www.giantbomb.com/api/game/3030-30615/',
  'id': 30615,
  'name': 'Gears of War 3',
  'site_detail_url': 'https://www.giantbomb.com/gears-of-war-3/3030-30615/'},
 {'api_detail_url': 'https://www.giantbomb.com/api/game/3030-21590/',
  'id': 21590,
  'name': 'Mass Effect 2',
  'site_detail_url': 'https://www.giantbomb.com/mass-effect-2/3030-21590/'},
 {'api_detail_url': 'https://www.giantbomb.com/api/game/3030-16637/',
  'id': 16637,
  'name': 'P.N.03',
  'site_detail_url': 'https://www.giantbomb.com/pn03/3030-16637/'},
 {'api_detail_url': 'https://www.giantbomb.com/api/game/3030-20464/',

In [5]:
#Function Testing

def GetGenre(game_id):##This gets the genre of the user-inputted game.
    key = '64ce54b0ab6eb483b1964ae7979124f96aa791e5'
    game_client = pb.GameClient(key, default_format = 'json')
    response = game_client.fetch(game_id)
    
    genre_list = []
    genre_list.append(response.results['genres'][0]['name'])
    return(genre_list)
    
GetGenre(48754)

['First-Person Shooter']

In [6]:

#The API does a search based on the user input, returning a ton of information about every game that matches.
#This method parses through the ids, and appends them to a list.
#In our main program, we take just the id of the first game that matches

def returnIds():

    user_game = input("Enter the name of a game you'd like to search for:")
    key = '64ce54b0ab6eb483b1964ae7979124f96aa791e5' ##Make this an input or encrypt it
    games_client = pb.GamesClient(key)
    response = games_client.quick_search(name = user_game)
    results = response.results
    
    name_list = []
    id_list = []
    
    for i in results:
        
        name_list.append(i['aliases'])
        id_list.append(i['id'])
    
    return(id_list)
returnIds()

Enter the name of a game you'd like to search for:Call of Duty 2


[2347, 16711]

In [51]:
#Main Program:
ids = returnIds()
try:
    recinfo = GetSimilarGames(ids)
    df = compileDF(recinfo)
    -
except(pb.exceptions.BadRequestException, IndexError, requests.exceptions.HTTPError, TypeError) as e:
    print("No really, game's not found. This is just the reccommendations for the last game:")
    pass---
df

Enter the name of a game you'd like to search for:jafiowfajwfeefao ffjifoaewfjiaweof jiofawe
game not found
No really, game's not found. This is just the reccommendations for the last game:


,Games,URLs
0,Destiny,https://www.giantbomb.com/destiny/3030-36067/
1,Resistance 3,https://www.giantbomb.com/resistance-3/3030-29...
2,Gunslinger Stratos,https://www.giantbomb.com/gunslinger-stratos/3...
